In [1]:
import re, os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import shutil
import xml.dom.minidom
import xml.etree.ElementTree as ET

### 矩阵自适应裁黑边

In [29]:
def change_size(img_delete_font):
    # pdb.set_trace()
    img = cv2.medianBlur(img_delete_font,5) #中值滤波，去除黑色边际中可能含有的噪声干扰
    imgray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    h ,w = imgray.shape
    img_1_h = np.ones((1, 1000)).astype(float)  # 截取中间区域height:1000
    img_w_1 = np.ones((1200, 1)).astype(float)   # 截取中间区域width:1200
    w_half = int(image.shape[1]/2-600)  # 截取中间区域width:1200——对应的left边距的坐标
    h_half = int(image.shape[0]/2-500)  # 截取中间区域height:1000——对应的top边距的坐标
#     print(h ,w, h_half, w_half)
    if (w - 2*w_half)==1200 and (h - 2*h_half)==1000: 
        img_row_merge = np.dot(img_1_h, imgray[h_half:-h_half,:]).astype(float)  # imgray[326:-326,:]
        img_col_merge = np.dot(imgray[:, w_half:-w_half],img_w_1).astype(float) 
    elif (w - 2*w_half)==1200 and (h - 2*h_half)>1000:  # 针对height==1001
        h_deta = (h - 2*h_half)-1000
        print('h_deta', h_deta)
        img_row_merge = np.dot(img_1_h, imgray[h_half:-h_half-h_deta,:]).astype(float)  # imgray[326:-326,:]
        img_col_merge = np.dot(imgray[:, w_half:-w_half],img_w_1).astype(float)
    elif (w - 2*w_half)>1200 and (h - 2*h_half)==1000:  # 针对width==1201
        w_deta = (w - 2*w_half)-1200
        print('w_deta', w_deta)
        img_row_merge = np.dot(img_1_h, imgray[h_half:-h_half,:]).astype(float)  # imgray[326:-326,:]
        img_col_merge = np.dot(imgray[:, w_half:-w_half-w_deta],img_w_1).astype(float)
    elif (w - 2*w_half)>1200 and (h - 2*h_half)>1000:  # 针对height==1001, width==1201
        h_deta = (h - 2*h_half)-1000
        w_deta = (w - 2*w_half)-1200
        print('h_deta:{}, w_deta:{}'.format(h_deta, w_deta))
        img_row_merge = np.dot(img_1_h, imgray[h_half:-h_half-h_deta,:]).astype(float)  # imgray[326:-326,:]
        img_col_merge = np.dot(imgray[:, w_half:-w_half-w_deta],img_w_1).astype(float)
    else :
        print('错误信息：{0}, {1}'.format(w_half, h_half))
        
    ratio = ((w + h) / 2 )*2.5  # 裁剪图像的固定阈值
    row_base_color = img_row_merge[0][0]  # 获得图像中黑色像素点对应值——row压缩后
    col_base_color = img_col_merge[0][0]  # 获得图像种黑色像素点对应值——col压缩后
    if row_base_color == 0 and col_base_color == 0:  # 排除图像底色并非黑色的
        pass
    else:
        print("row_base_color:{} \ncol_base_color:{}".format(row_base_color, col_base_color))
        img_row_merge = img_row_merge - row_base_color  # 减去基线的像素值——强制转为黑色0——row
        img_col_merge = img_col_merge - col_base_color  # 减去基线的像素值——强制转为黑色0——col
    
    for i in range(w):
        if img_row_merge[0][i] > ratio:
            left = i
            break

    for i in range(w-1,-1,-1):
        if img_row_merge[0][i] > ratio:
            right = i
            break

    for j in range(h):
        if img_col_merge[j][0] > ratio:
            up = j
            break

    for j in range(h-1,-1,-1):
        if img_col_merge[j][0] > ratio:
            down = j
            break
    
    return left, right, up, down

In [30]:
read_file = r'/Users/edz/Documents/yandi_data/taishi_loading/train_sets/train_image_02_裁黑边/032a8687cb7b9d835a40c919faf9c43e.jpg'
image=cv2.imread(read_file) #读取图片 image_name应该是变量
left,right,up,down = change_size(image)
print(left,right,up,down)
image_cropp = image[up:down, left:right]

h_deta:1, w_deta:1
row_base_color:35000.0 
col_base_color:42000.0
500 2091 69 1659


### xml文件打标框根据图片等比例缩放

In [33]:
def xml_location_ratio_func(xml_path, ratio_width, ratio_height, img_height_new, img_width_new, left, up):
    # 定义图片的缩放比率——图片的缩放比例和打标框的缩放比例是相等的
    DOMTree = xml.dom.minidom.parse(xml_path)  # 读入xml文件
    collection = DOMTree.documentElement
    ########## （1）读入打标框的size，并根据缩放比进行修改
    sizenamelist = collection.getElementsByTagName('size')  # 第一层——size
    for sizes in sizenamelist:
        width_set = sizes.getElementsByTagName('width')  # 第二层——width
        height_set = sizes.getElementsByTagName('height')  # 第二层——height
        # 重新赋值——打标框的尺寸
        width_set[0].childNodes[0].data = int(img_width_new)
#         print('图像size_width_new', width_set[0].childNodes[0].data)
        height_set[0].childNodes[0].data = int(img_height_new)
#         print('图像size_height_new', height_set[0].childNodes[0].data)
    ########## （2）读入打标框的位置坐标，并根据缩放比进行修改
    filenamelist = collection.getElementsByTagName('object')  # 第一层——object
    for objects in filenamelist:
        bndboxlist = objects.getElementsByTagName('bndbox')  # 第二层
        for items in bndboxlist:
            xmin = items.getElementsByTagName('xmin')  # 第三层——xmin
            ymin = items.getElementsByTagName('ymin')  # 第三层——ymin
            xmax = items.getElementsByTagName('xmax')  # 第三层——xam
            ymax = items.getElementsByTagName('ymax')  # 第三层——ymax  
            # 计算新的打标框width和hight
            xml_width_new = (float(xmax[0].childNodes[0].data) - float(xmin[0].childNodes[0].data))*ratio_width
#             print('xml打标框的width_new:{}'.format(xml_width_new))
            xml_hight_new = (float(ymax[0].childNodes[0].data) - float(ymin[0].childNodes[0].data))*ratio_height
#             print('xml打标框的height_new:{}'.format(xml_hight_new))
            # 重新赋值打标框
            xmin[0].childNodes[0].data = round((float(xmin[0].childNodes[0].data) - left), 2)  # 重新赋值 x_left_new = x_left_old - img_left
#             print('xmin_new' ,xmin[0].childNodes[0].data)
            ymin[0].childNodes[0].data = round((float(ymin[0].childNodes[0].data) - up), 2)  # 重新赋值 y_up_new = y_up_old - img_left
#             print('ymin_new' ,ymin[0].childNodes[0].data)
            xmax[0].childNodes[0].data = round((float(xmin[0].childNodes[0].data) + xml_width_new), 2)  # 重新赋值 x_right_new = x_left_new + width_new
#             print('xmax_new' ,xmax[0].childNodes[0].data)
            ymax[0].childNodes[0].data = round((float(ymin[0].childNodes[0].data) + xml_hight_new), 2)  # 重新赋值 y_down_new = y_up_new + height_new
#             print('ymax_new' ,ymax[0].childNodes[0].data)
    return DOMTree

In [ ]:
# 读入图像文件，并调用裁黑边函数——裁黑边，之后保存新图像
target_folder_path = r'/Users/edz/Documents/yandi_data/taishi_loading/train_sets/train_image_02_裁黑边/'  # 存放裁剪后图像
folder_path = r'/Users/edz/Documents/yandi_data/taishi_loading/train_sets/train_image_02/'  # 原图像路径
# folder_path = r'/Users/edz/Documents/yandi_data/taishi_loading/load_image/原始TXT表/第二批七种病灶/id12_玻璃膜疣/'  # 原图像路径
# 定义xml文件存放路径
target_xml_path = '/Users/edz/Documents/yandi_data/taishi_loading/train_sets/train_xml_02_等比例缩放/'
xml_path = r'/Users/edz/Documents/yandi_data/taishi_loading/test_sets/test_result_第二批七种病灶/xml_results/second_test_part02_xml/'

for file in os.listdir(folder_path):
    if file.endswith('.jpg'):
        file_path = os.path.join(folder_path, file)
#         print(file_path)
        image=cv2.imread(file_path) #读取图片 image_name应该是变量
        left, right, up, down = change_size(image) # 调用裁黑边函数，完成裁黑边操作
        image_cropp = image[up:down, left:right]
        cv2.imwrite(target_folder_path+file, image_cropp)  # 保存裁剪后的图像
        # 原图像的尺寸和裁剪后新图像的尺寸
        img_height_old, img_width_old= image.shape[0], image.shape[1]  # height, width
#         print("图像height_old:{}, width_old:{}".format(img_height_old,img_width_old))
        img_height_new, img_width_new= image_cropp.shape[0], image_cropp.shape[1]  # height, width
#         print("图像height_new:{}, width_new:{}".format(img_height_new,img_width_new))
        ratio_width = round(img_width_new/img_width_old, 2)
        ratio_height = round(img_height_new/img_height_old, 2)
#         print('ratio_width:{}'.format(ratio_width))
#         print('ratio_height:{}'.format(ratio_height))
        
        xml_file_path = xml_path + file[:-4] + '.xml'
        if not os.path.exists(xml_file_path):
            pass
        else:
            # 调用xml文件的打标框位置转换函数得到等比例缩放后的xml文件——用于下一步写入
            DOMTree_new = xml_location_ratio_func(xml_file_path, ratio_width, ratio_height, img_height_new, img_width_new, left, up)
            #定义输出的xml文件路径和名称
            temp_xmlfile = target_xml_path + file[:-4] + '.xml'
            try:
                with open(temp_xmlfile,'w',encoding='UTF-8') as fh:
             # 4.writexml()第一个参数是目标文件对象，第二个参数是根节点的缩进格式，第三个参数是其他子节点的缩进格式，
             # 第四个参数制定了换行格式，第五个参数制定了xml内容的编码。
                    DOMTree_new.writexml(fh)
#                     print('写入xml OK!')
            except Exception as err:
                print('错误信息：{0}'.format(err))

In [ ]:
# # 定义xml文件存放路径
# xml_path = r'/Users/edz/Documents/yandi_data/taishi_loading/test_sets/test_result_第二批七种病灶/xml_results/second_test_part02_xml/b32ebc1509b85fe24222bcac98dd3d64.xml'
# # target_xml_path = '/Users/edz/Desktop/'
# ################ 文件保存
# # 调用xml文件的打标框位置转换函数得到等比例缩放后的xml文件——用于下一步写入
# DOMTree_new = xml_location_ratio_func(xml_path, ratio_width, ratio_height, 1555, 1923)
# #定义输出的xml文件路径和名称
# temp_xmlfile = '/Users/edz/Desktop/' + 'test_黄斑前膜.xml'
# try:
#     with open(temp_xmlfile,'w',encoding='UTF-8') as fh:
#  # 4.writexml()第一个参数是目标文件对象，第二个参数是根节点的缩进格式，第三个参数是其他子节点的缩进格式，
#  # 第四个参数制定了换行格式，第五个参数制定了xml内容的编码。
#         DOMTree_new.writexml(fh)
#         print('写入xml OK!')
# except Exception as err:
#     print('错误信息：{0}'.format(err))

In [40]:
# # 读入图像文件，并调用裁黑边函数——裁黑边，之后保存新图像
# target_folder_path = r'/Users/edz/Documents/yandi_data/taishi_loading/train_sets/train_image_02_裁黑边/'  # 存放裁剪后图像
# folder_path = r'/Users/edz/Documents/yandi_data/taishi_loading/train_sets/train_image_02/'  # 原图像路径
# # folder_path = r'/Users/edz/Documents/yandi_data/taishi_loading/load_image/原始TXT表/第二批七种病灶/id12_玻璃膜疣/'  # 原图像路径
# # 定义xml文件存放路径
# target_xml_path = '/Users/edz/Documents/yandi_data/taishi_loading/train_sets/train_xml_02_等比例缩放/'
# xml_path = r'/Users/edz/Documents/yandi_data/taishi_loading/test_sets/test_result_第二批七种病灶/xml_results/second_test_part02_xml/'

# for file in os.listdir(folder_path):
#     if file.endswith('.jpg'):
#         file_path = os.path.join(folder_path, file)
# #         print(file_path)
#         image = cv2.imread(file_path) #读取图片 image_name应该是变量
#         if file in fix_list:
#             left, right, up, down = change_size(image) # 调用裁黑边函数，完成裁黑边操作
#             image_cropp = image[up:down, left:right]
#             cv2.imwrite(target_folder_path+file, image_cropp)  # 保存裁剪后的图像
# #         # 原图像的尺寸和裁剪后新图像的尺寸
#             img_height_old, img_width_old= image.shape[0], image.shape[1]  # height, width
#     #         print("图像height_old:{}, width_old:{}".format(img_height_old,img_width_old))
#             img_height_new, img_width_new= image_cropp.shape[0], image_cropp.shape[1]  # height, width
#     #         print("图像height_new:{}, width_new:{}".format(img_height_new,img_width_new))
#             ratio_width = round(img_width_new/img_width_old, 2)
#             ratio_height = round(img_height_new/img_height_old, 2)
#     #         print('ratio_width:{}'.format(ratio_width))
#     #         print('ratio_height:{}'.format(ratio_height))

#             xml_file_path = xml_path + file[:-4] + '.xml'
#             if not os.path.exists(xml_file_path):
#                 pass
#             else:
#                 # 调用xml文件的打标框位置转换函数得到等比例缩放后的xml文件——用于下一步写入
#                 DOMTree_new = xml_location_ratio_func(xml_file_path, ratio_width, ratio_height, img_height_new, img_width_new, left, up)
#                 #定义输出的xml文件路径和名称
#                 temp_xmlfile = target_xml_path + file[:-4] + '.xml'
#                 try:
#                     with open(temp_xmlfile,'w',encoding='UTF-8') as fh:
#                  # 4.writexml()第一个参数是目标文件对象，第二个参数是根节点的缩进格式，第三个参数是其他子节点的缩进格式，
#                  # 第四个参数制定了换行格式，第五个参数制定了xml内容的编码。
#                         DOMTree_new.writexml(fh)
#     #                     print('写入xml OK!')
#                 except Exception as err:
#                     print('错误信息：{0}'.format(err))

row_base_color:7000.0 
col_base_color:8400.0
row_base_color:1440.0 
col_base_color:1896.0
row_base_color:53838.0 
col_base_color:18572.0
row_base_color:2633.0 
col_base_color:2289.0
row_base_color:1693.0 
col_base_color:1909.0
row_base_color:1873.0 
col_base_color:2265.0
row_base_color:1523.0 
col_base_color:1455.0
row_base_color:45000.0 
col_base_color:54000.0
row_base_color:1458.0 
col_base_color:1708.0
row_base_color:7640.0 
col_base_color:87283.0
row_base_color:25699.0 
col_base_color:27309.0
h_deta 1
row_base_color:1050.0 
col_base_color:1552.0
row_base_color:3888.0 
col_base_color:5961.0
row_base_color:1548.0 
col_base_color:2062.0
row_base_color:2854.0 
col_base_color:2721.0
row_base_color:6.0 
col_base_color:52778.0
row_base_color:1373.0 
col_base_color:1798.0
row_base_color:1166.0 
col_base_color:1568.0
row_base_color:13296.0 
col_base_color:152181.0
row_base_color:6896.0 
col_base_color:0.0
row_base_color:35000.0 
col_base_color:42000.0
row_base_color:66335.0 
col_base_color:

In [ ]:
os.path.splitext(file_path)